# Multimodel use cases

In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform\
                                        gitpython \
                                        magika

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PROJECT_ID = "inner-orb-434419-b6"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.generative_models import GenerativeModel, Image, Part


In [ ]:
multimodal_model = GenerativeModel("gemini-1.5-pro-001")

multimodal_model_flash = GenerativeModel("gemini-1.5-flash-001")

In [ ]:
import http.client
import typing
import urllib.request
import IPython.display

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_video(content: str | Image | Part):
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=350))

## Understanding the text


In [ ]:
question = "What are some must-try local dishes in Mountain View, CA?"
prompt = """
Given these popular local dishes , recommend some restaurants
or food spots in Mountain View where visitors can enjoy these authentic flavors.
"""

contents = [question, prompt]
response = multimodal_model.generate_content(contents)
display(IPython.display.Markdown(response.text))


## Must-Try Local Dishes in Mountain View, CA & Where to Find Them:

While Mountain View doesn't have its own distinct cuisine, it boasts incredible diversity and quality in its culinary scene.  Here are some "must-try" dishes reflecting that variety, along with specific restaurant recommendations:

**1. World-Class Vietnamese:**  Mountain View is practically *famous* for Vietnamese food.

* **Phở:**  Aromatic broth, rice noodles, meat, and fresh herbs.
    * **Phở Hòa Noodle Soup:** Renowned for their rich, flavorful broth (multiple locations)
    * **Pho Hoa Pasteur:**  Another local favorite, known for authentic Hanoi-style phở 
* **Bún Chả Hà Nội:** Vermicelli noodles, grilled pork, fresh herbs, dipping sauce. 
    * **Hanoi Bistro:**  Highly praised for their bún chả and other northern Vietnamese specialties
* **Bánh Mì:**  Crusty baguette sandwiches with various fillings (grilled meats, pate, pickled veggies).
    * **Lee's Sandwiches:** A classic choice for quick, delicious bánh mì (multiple locations)
    * **Xanh Vietnamese Restaurant:** Upscale option with unique bánh mì creations

**2. Silicon Valley's Global Flavors:**  Mountain View's tech scene attracts cuisines from around the world.

* **Ramen:**  Rich, flavorful Japanese noodle soup with various toppings.
    * **Ramen Nagi:**  Customizable ramen with a wide selection of broths and toppings 
    * **Santouka Ramen:**  Known for their creamy, pork-based tonkotsu ramen
* **Korean BBQ:**  Grill your own marinated meats at your table, served with banchan (small side dishes).
    * **Gen Korean BBQ House:** Popular spot with all-you-can-eat options
    * **Chungdam Korean Fusion Bistro:** Modern take on Korean classics, including BBQ
* **Mediterranean:**  Fresh, flavorful dishes like falafel, hummus, shawarma.
    * **Dishdash:** Popular spot for Middle Eastern and Mediterranean cuisine, try the mixed grill 
    * **Falafel Stop:**  As the name suggests, great spot for delicious falafel and shawarma

**3. Californian Cuisine:**  Embrace the fresh, local ingredients.

* **Farm-to-Table:**  Restaurants sourcing ingredients from nearby farms.
    * **The Village Pub:** Michelin-starred restaurant with an emphasis on seasonal menus
    * **Scratch:**  Casual eatery using locally sourced ingredients for their dishes
* **Seafood:**  Being close to the Pacific, Mountain View offers fresh seafood options.
    * **The Fish Market:**  Wide variety of fresh seafood, great for a special occasion
    * **Cascal:**  Modern Mexican restaurant with a focus on seafood dishes


**Beyond specific dishes:**

* **Castro Street:**  The heart of Mountain View's dining scene with diverse options.
* **Farmers' Markets:** Experience the local bounty, often with food vendors (Sundays on California St.)

This is just a starting point!  Don't hesitate to explore and discover your own Mountain View culinary favorites.

## Document Summarization

In [ ]:
import vertexai

from vertexai.generative_models import GenerativeModel, Part

model = GenerativeModel("gemini-1.5-flash-001")

prompt = """
You are a very professional document summarization specialist.
Please summarize the given document.
"""

pdf_file = Part.from_uri(
    uri="gs://part-c-project/somatosensory.pdf",
    mime_type="application/pdf",
)
contents = [pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

This document from Wikibooks discusses the anatomy of the somatosensory system which provides information about touch, temperature, pressure, pain, and position. The system is made up of cutaneous receptors located in the skin and receptors found in muscles and joints. The document outlines the different types of cutaneous receptors, describing their characteristics and functions. It also explains the structure and function of muscle spindles, the stretch receptors found in muscles. Finally, the document discusses the various types of joint receptors and their roles in providing information about the position and movement of joints.


## Image Understanding

In [ ]:
response = model.generate_content(contents=[...], stream = True)

In [ ]:
import vertexai

from vertexai.generative_models import GenerativeModel, Part

# TODO(developer): Update project_id and location
vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

image_file = Part.from_uri(
    "gs://part-c-project/square face.png", "image/jpeg"
)

# Query the model
response = model.generate_content([image_file, "what is this image?"])
print(response.text)

This is an image of a young woman with dark hair and brown eyes. She is wearing a white sweater and has her hair styled in a casual updo. The background is a light brown color. The photo is taken from the shoulders up, with the focus on the woman's face.


## Video Understanding
Generating a video description

Click here to watch video: https://drive.google.com/file/d/1B5YP3vd14xLALy0mMZG_NMh4htzZdyjP/view?usp=sharing

In [ ]:
prompt = """
What is shown in this video?
Which movie/series is this video from?
What are the top 5 cartoons similar to this?
"""
video = Part.from_uri(
    uri="gs://part-c-project/SampleVideo_1280x720_2mb.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = multimodal_model.generate_content(contents)

display_content_as_video(video)
display(IPython.display.Markdown(responses.text))

The video shows a large, rotund rabbit waking up from a nap and taking in a deep breath of fresh air. It appears to be from a movie or series with 3D animation. 

Unfortunately, it is impossible to determine the exact movie or series from such a short clip.

However, based on the art style, here are 5 cartoons that bear a resemblance and could be considered similar:

1. **Over the Hedge:** This Dreamworks film features a similar animation style with anthropomorphic animals.
2. **Open Season:**  Another Sony Pictures animation with talking animals and a bright, playful visual style.
3. **Surf's Up:**  Although about penguins, this film shares a similar vibrancy and comedic tone.
4. **Cloudy with a Chance of Meatballs:**  This film is visually comparable with its exaggerated characters and colorful world.
5. **The Adventures of Tintin:**  While not exclusively focused on animals, Spielberg's Tintin film utilizes similar motion capture animation for its characters. 

Without more context, pinpointing the exact source of the clip is difficult. 


## Audio Understanding

In [ ]:
  import vertexai
  from vertexai.generative_models import GenerativeModel, Part

  model = GenerativeModel("gemini-1.5-flash-001")

  prompt = """
  1. Please provide a comprehensive summary of this audio recording.
  2. Focus on identifying the key points, main themes, and any critical information discussed.
  3. Assess the emotional tone and sentiment throughout the audio.
  4. Can you tell who is the speaker?

"""

  audio_file_uri = "gs://part-c-project/NzMxMDUyMTcyNzMxMTMz_t2_2foJnPt274.mp3"
  audio_file = Part.from_uri(audio_file_uri, mime_type="audio/mpeg")

  contents = [audio_file, prompt]

  response = model.generate_content(contents)
  print(response.text)

This audio recording is a news segment that features an interview with Dr. Alice Krippen, a medical researcher who has made a breakthrough in cancer treatment. The interview starts by highlighting the significant medical advancements throughout history, comparing them to Dr. Krippen's work. 

Dr. Krippen explains that her research involves reprogramming a measles virus to work for the body rather than against it. She uses a car analogy to illustrate the process, comparing the virus to a speeding car driven by a "bad man" causing damage, and then replacing the "bad man" with a "cop" to change the picture. This demonstrates how they have engineered the virus to become beneficial. 

The interviewer then inquires about the number of patients treated and the results. Dr. Krippen reveals that they have conducted 10,009 clinical trials on humans, and all of them have resulted in cancer being cured. 

The tone of the interview is incredibly positive and optimistic, reflecting the groundbreakin

## Code Generation

In [1]:
!pip install --upgrade openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [5]:
# Function to securely input API keys
def get_api_key(api_name):
    from google.colab import userdata
    key = userdata.get(api_name)
    if key is None:
        raise ValueError(f"{api_name} not found. Please set it in Colab.")
    return key

# Set up OpenAI (GPT-4) API
openai_client = OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

# Test GPT-4
def test_gpt4():
    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": "Say hello!"}]
    )
    return response.choices[0].message.content

In [6]:
import openai
from openai import OpenAI
import re

# Assuming openai_client is already set up from the previous step
client = OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

def generate_code_gpt4(prompt):
    """
    Generate code based on a natural language prompt using GPT-4.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates Python code based on natural language descriptions. Provide only the Python code without any additional explanation."},
            {"role": "user", "content": f"Generate Python code for the following task: {prompt}"}
        ]
    )
    return response.choices[0].message.content

def extract_python_code(text):
    """
    Extract Python code from text, removing any markdown formatting.
    """
    # Remove markdown code block syntax
    code = re.sub(r'```python|```', '', text, flags=re.IGNORECASE)
    return code.strip()

# Example usage
prompt = "Create a function that takes a list of integers and returns a new list containing only the prime numbers."

generated_code = generate_code_gpt4(prompt)
print("Generated Code:")
print(generated_code)

# Extract and execute the generated code
extracted_code = extract_python_code(generated_code)
print("\nExtracted Code:")
print(extracted_code)

exec(extracted_code)

# Test the generated function
test_list = [1, 2, 3, 4, 5, 11, 13, 17, 19, 23]
result = filter_prime_numbers(test_list)  # This function name assumes GPT-4 used this name; adjust if different
print(f"\nTesting the generated function with {test_list}")
print(f"Result: {result}")


Generated Code:
```python
def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
      
    return True


def filter_prime_numbers(lst):
    return [num for num in lst if is_prime(num)]
```

Extracted Code:
def is_prime(n):
    if n <= 1:
        return False

    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
      
    return True


def filter_prime_numbers(lst):
    return [num for num in lst if is_prime(num)]

Testing the generated function with [1, 2, 3, 4, 5, 11, 13, 17, 19, 23]
Result: [2, 3, 5, 11, 13, 17, 19, 23]


## Refactoring Code

In [7]:
import openai
import re

# Setup OpenAI client
client = openai.OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

def generate_refactoring_suggestions(code, description):
    """
    Generate refactoring suggestions based on a description of the desired improvements.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that refactors Python code based on given descriptions. Provide only the Python code without any additional explanation."},
            {"role": "user", "content": f"Refactor the following Python code: {code}. {description}"}
        ]
    )
    return response.choices[0].message.content

def extract_python_code(text):
    """
    Extract Python code from text, removing any markdown formatting.
    """
    code = re.sub(r'```python|```', '', text, flags=re.IGNORECASE)
    return code.strip()

# Example existing code
existing_code = """
def calculate_sum(lst):
    result = 0
    for num in lst:
        result += num
    return result
"""

# Desired refactoring: use list comprehension and improve performance if applicable
description = "Refactor this code to use list comprehension and enhance performance."

# Generate refactoring suggestions
refactoring_suggestions = generate_refactoring_suggestions(existing_code, description)
print("Refactoring Suggestions:")
print(refactoring_suggestions)

# Extract and possibly execute the refactored code
refactored_code = extract_python_code(refactoring_suggestions)
print("\nRefactored Code:")
print(refactored_code)

# Optionally, you can test the refactored code by executing it
exec(refactored_code)
test_list = [1, 2, 3, 4, 5]
result = calculate_sum(test_list)
print(f"\nTesting the refactored function with {test_list}")
print(f"Result: {result}")


Refactoring Suggestions:
def calculate_sum(lst):
    return sum(num for num in lst)

Refactored Code:
def calculate_sum(lst):
    return sum(num for num in lst)

Testing the refactored function with [1, 2, 3, 4, 5]
Result: 15


## Content Generation

In [8]:
import openai

# Setup OpenAI client
client = openai.OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

def generate_content(prompt, content_type, max_tokens=150):
    """
    Generate content based on a natural language prompt using GPT-4.
    Parameters:
        prompt (str): The content prompt describing what to write about.
        content_type (str): Type of content, e.g., 'article', 'blog post', 'social media'.
        max_tokens (int): Maximum number of tokens to generate.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"You are a creative assistant that generates {content_type} based on natural language prompts."},
            {"role": "user", "content": f"Generate a {content_type} about the following topic: {prompt}"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Example usages
article_prompt = "the impact of climate change on global agriculture"
blog_post_prompt = "top 5 tips for beginner yoga practitioners"
social_media_prompt = "exciting announcement about our new product launch next week!"

# Generate different types of content
article_content = generate_content(article_prompt, 'article', max_tokens=500)
blog_content = generate_content(blog_post_prompt, 'blog post', max_tokens=300)
social_media_content = generate_content(social_media_prompt, 'social media post', max_tokens=100)

print("Generated Article Content:")
print(article_content)
print("\nGenerated Blog Post Content:")
print(blog_content)
print("\nGenerated Social Media Content:")
print(social_media_content)


Generated Article Content:
Title: The Impact of Climate Change on Global Agriculture

Introduction 

Climate change, a phenomenon marked by rising global temperatures, desertification, frequent occurrence of extreme weather events, and sea-level surges, is reshaping the overall schematic of our world. In this dynamic planetary rearrangement, no field—literally and figuratively— remains immune, especially not the sector most dependent on Mother Nature: Agriculture. This article explores the multifaceted impact of climate change on global agriculture, altering everything from crop yields to distribution networks and posing significant challenges for future food security.

Climate Change's Impact On Crop Cultivation

Agriculture is an industry highly contingent on stable weather conditions and favorable climates. With climate change, this stability is under severe threat. Rising temperatures are often associated with increased evapotranspiration (the combination of evaporation and plant t

## Creative Arts using OpenAI GPT-4

In [9]:
import openai

# Setup OpenAI client
client = openai.OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

def generate_creative_content(prompt, content_type, max_tokens=200):
    """
    Generate creative content based on a natural language prompt using GPT-4.
    Parameters:
        prompt (str): The content prompt describing the creative task.
        content_type (str): Type of creative content, e.g., 'story', 'script', 'music composition'.
        max_tokens (int): Maximum number of tokens to generate.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"You are an artistic assistant that generates {content_type} based on natural language prompts."},
            {"role": "user", "content": f"Generate a {content_type} based on the following idea: {prompt}"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Example usages
story_prompt = "a young wizard discovering their powers in a modern city"
script_prompt = "a dialogue between two old friends who accidentally meet after 20 years at an airport"
music_composition_prompt = "compose a short piece in the style of Baroque music for a piano"

# Generate different types of creative content
story_content = generate_creative_content(story_prompt, 'story', max_tokens=500)
script_content = generate_creative_content(script_prompt, 'script', max_tokens=500)
music_composition_content = generate_creative_content(music_composition_prompt, 'music composition', max_tokens=300)

print("Generated Story Content:")
print(story_content)
print("\nGenerated Script Content:")
print(script_content)
print("\nGenerated Music Composition Content:")
print(music_composition_content)


Generated Story Content:
Title: The Awakening of Urban Sorcery

In the bustling modern city of New York, where skyscrapers kissed the sky amidst the bustling crowd, chaotic honks, and flashing neon lights, lived an unassuming teenager named Max. Everyone knew Max as an introverted, yet brilliant gamer and technology enthusiast. However, one unexpected day, Max's ordinary life would take a spellbinding turn. He would discover he was not as ordinary as he perceived himself to be; he was, in fact, a wizard.

Max’s revelation came in a rather inadvertent manner. One day, while he was in a heated gaming competition, frustration rushed over him as he was on the verge of losing. In his frustration, he unconsciously wished the power would cut. To his absolute surprise and horror, not a moment later, there was a sudden, unexplained blackout across the entire block. With his heart pounding and mind swirling with questions, he started to suspect that the blackout was not a coincidence at all.

Ov

## Healthcare


In [14]:
import openai

# Setup OpenAI client
client = openai.OpenAI(api_key=get_api_key('OPENAI_API_KEY'))

def generate_medical_summary(research_text, max_tokens=300):
    """
    Summarize medical research articles using GPT-4.
    Parameters:
        research_text (str): Text of the medical research article.
        max_tokens (int): Maximum number of tokens for the summary.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes medical research articles."},
            {"role": "user", "content": f"Summarize this medical research: {research_text}"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

def generate_care_plan(patient_details, max_tokens=300):
    """
    Generate a patient care plan based on medical history and current symptoms using GPT-4.
    Parameters:
        patient_details (str): Description of the patient's medical history and symptoms.
        max_tokens (int): Maximum number of tokens for the care plan.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant that generates patient care plans based on their medical history and symptoms."},
            {"role": "user", "content": f"Create a care plan for this patient: {patient_details}"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Example usage
research_text = "A recent study on the effects of a new diabetes medication shows promising results in reducing blood sugar levels without significant side effects."
patient_details = "Patient is a 58-year-old male with a history of Type 2 Diabetes and hypertension. Recent symptoms include increased thirst and frequent urination."

# Generate medical summary
medical_summary = generate_medical_summary(research_text)
print("Medical Research Summary:")
print(medical_summary)

# Generate patient care plan
care_plan = generate_care_plan(patient_details)
print("\nPatient Care Plan:")
print(care_plan)


Medical Research Summary:
The recent research study focuses on a new diabetes medication that has demonstrated positive outcomes. The results revealed that this new drug is capable of successfully decreasing blood sugar levels. Crucially, it also causes minimal major side effects, indicating it could represent a beneficial new treatment solution for diabetes patients.

Patient Care Plan:
Patient Information: 
- 58-year-old male 
- Past medical history of Type 2 Diabetes and Hypertension 
- Recent symptoms: increased thirst and frequent urination

**Patient Care Plan**

1. **Problem identification:** 
   High blood sugar (hyperglycemia) signs represented by increased thirst and frequent urination indicates uncontrolled diabetes condition. Hypertension (high blood pressure) may complicate things if not properly managed.

2. **Goal establishment:**
   Primary aims include getting patient's blood sugar levels under control, managing blood pressure, and maintaining a healthy lifestyle to av